# Milk Yield Forecasting Notebook
This notebook loads daily cow milking records, cleans the data, aggregates total daily milk yield, and fits a simple time-series forecasting model.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from statsmodels.tsa.statespace.sarimax import SARIMAX

%matplotlib inline

plt.rcParams['figure.figsize'] = (12, 5)
print("Libraries imported.")

ModuleNotFoundError: No module named 'statsmodels'

## Load and Combine Daily Records

In [ ]:
# Adjust this path if your notebook is not inside the `notebooks/` folder
data_dir = Path("..") / "data"

# Look for all daily cow record CSV files
csv_files = sorted(data_dir.glob("daily_cow_records_*.csv"))
print(f"Found {len(csv_files)} CSV files:")
for f in csv_files:
    print(" -", f.name)

if not csv_files:
    raise FileNotFoundError("No files matching 'daily_cow_records_*.csv' were found in the data/ folder.")

## Clean Column Names and Dates

In [ ]:
def clean_column_name(col: str) -> str:
    """Clean weird encodings in column names, e.g. 'Cow+AF8-ID' -> 'Cow_ID'."""
    col = str(col)
    col = col.replace("+AF8-", "_").replace("+AF8", "_")
    col = col.strip()
    return col

def clean_date_value(val) -> str:
    """Clean weird encodings in dates like '2025+AC0-11+AC0-18' -> '2025-11-18'."""
    s = str(val)
    # Replace encoded hyphen patterns
    s = s.replace("+AC0-", "-").replace("+AC0", "-")
    # Sometimes extra characters may appear; keep only first 10 chars (YYYY-MM-DD)
    if len(s) >= 10:
        s = s[:10]
    return s

# Load and combine all CSVs
dfs = []
for path in csv_files:
    tmp = pd.read_csv(path)
    tmp["source_file"] = path.name
    dfs.append(tmp)

df_raw = pd.concat(dfs, ignore_index=True)
df_raw.columns = [clean_column_name(c) for c in df_raw.columns]

print("Columns after cleaning:")
print(df_raw.columns.tolist())
df_raw.head()

## Create a Proper Date Column

In [ ]:
# Try direct datetime conversion first
df = df_raw.copy()

if "Date" not in df.columns:
    raise KeyError("Expected a 'Date' column in the data. Found columns: " + ", ".join(df.columns))

df["date_raw"] = df["Date"]

# First attempt: direct parse
df["date"] = pd.to_datetime(df["Date"], errors="coerce")

# If many NaT values, apply cleaning function and parse again
nat_ratio = df["date"].isna().mean()
print(f"NaT ratio after first parse: {nat_ratio:.2%}")

if nat_ratio > 0.2:
    print("Applying custom date cleaning...")
    df["date_clean_str"] = df["Date"].apply(clean_date_value)
    df["date"] = pd.to_datetime(df["date_clean_str"], errors="coerce")

print("Sample of original vs parsed dates:")
print(df[["Date", "date"]].head())

# Drop rows where date could not be parsed
df = df.dropna(subset=["date"]).copy()
df.head()

## Filter Milking Records and Aggregate Daily Totals

In [ ]:
# Try to identify the relevant columns
# We expect something like: 'Activity', 'Milk_Liters', 'Cow_ID'
cols_lower = {c.lower(): c for c in df.columns}

activity_col = cols_lower.get("activity")
milk_col = None
for key, col in cols_lower.items():
    if "milk" in key and "liter" in key:
        milk_col = col
        break

cow_col = None
for key, col in cols_lower.items():
    if "cow" in key and "id" in key:
        cow_col = col
        break

print("Using columns:")
print(" Activity:", activity_col)
print(" Milk:", milk_col)
print(" Cow ID:", cow_col)

if activity_col is None or milk_col is None:
    raise KeyError("Could not identify Activity or Milk_Liters columns. Check your CSV headers.")

# Keep only milking rows
df_milk = df[df[activity_col].astype(str).str.lower() == "milking"].copy()

# Ensure milk column is numeric
df_milk[milk_col] = pd.to_numeric(df_milk[milk_col], errors="coerce")

# Aggregate total milk per day
daily_total = (
    df_milk.groupby("date")[milk_col]
    .sum()
    .sort_index()
)

print("Daily total milk yield (first 10 days):")
print(daily_total.head(10))

# Optional: aggregate per cow per day (for later analysis)
if cow_col is not None:
    cow_daily = (
        df_milk.groupby(["date", cow_col])[milk_col]
        .sum()
        .reset_index()
    )
else:
    cow_daily = None

daily_total.plot()
plt.title("Total Daily Milk Yield")
plt.xlabel("Date")
plt.ylabel("Milk (liters)")
plt.show()

## Prepare Time Series for Forecasting

In [ ]:
# Ensure the index is a proper daily frequency time series
ts = daily_total.asfreq("D")

print("Time series length:", len(ts))
print("Start:", ts.index.min(), "End:", ts.index.max())

# Simple missing value handling: forward-fill then back-fill
ts = ts.ffill().bfill()

ts.plot()
plt.title("Total Daily Milk Yield (with daily frequency)")
plt.xlabel("Date")
plt.ylabel("Milk (liters)")
plt.show()

## Train-Test Split and SARIMAX Forecast

In [ ]:
# Use the last 7 days as test set, if we have enough data
if len(ts) < 30:
    print("Warning: less than 30 days of data. Forecast results may not be reliable.")
    
test_steps = min(7, max(1, len(ts) // 4))  # at least 1 step, up to a quarter of the data
train = ts.iloc[:-test_steps]
test = ts.iloc[-test_steps:]

print(f"Train size: {len(train)}, Test size: {len(test)}")

# Fit a simple SARIMAX model (you can experiment with these orders)
model = SARIMAX(train, order=(1, 1, 1), seasonal_order=(1, 1, 1, 7))
results = model.fit(disp=False)
print(results.summary())

# Forecast
forecast_res = results.get_forecast(steps=test_steps)
forecast_mean = forecast_res.predicted_mean
forecast_ci = forecast_res.conf_int()

# Plot
plt.figure(figsize=(12, 5))
plt.plot(train.index, train, label="Train")
plt.plot(test.index, test, label="Test", marker="o")
plt.plot(forecast_mean.index, forecast_mean, label="Forecast", marker="o")
plt.fill_between(forecast_ci.index,
                 forecast_ci.iloc[:, 0],
                 forecast_ci.iloc[:, 1],
                 alpha=0.2)
plt.title("Milk Yield Forecast vs Actual")
plt.xlabel("Date")
plt.ylabel("Milk (liters)")
plt.legend()
plt.show()

## Simple Forecast Evaluation

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Align forecast and test
common_index = test.index.intersection(forecast_mean.index)
y_true = test.loc[common_index]
y_pred = forecast_mean.loc[common_index]

mae = mean_absolute_error(y_true, y_pred)
rmse = mean_squared_error(y_true, y_pred, squared=False)

print(f"MAE: {mae:.3f}")
print(f"RMSE: {rmse:.3f}")